# Integrating AWS API Gateway with Kafka on MSK

## Introduction

We will build a **Kafka REST Proxy integration**, which provides a RESTful interface to a Kafka cluster. This makes it easy to produce and consume messages, view the state of a cluster, or perform administrative actions without using native Kafka protocols or clients.

## Kafka REST Proxy

To get started go to the **API Gateway console** and select one of your previously created APIs. Your API should have a `{proxy+}` resource.

For **Integration type** select HTTP Proxy. HTTP proxy integration is a simple, yet powerful way of building APIs that allow web applicaitons to access multiple resources on the integrated HTTP endpoint. In HTTP proxy integration, API Gateway simply passes client-submitted method requests to the backend. In turn the backend HTTP endpoint parses the incoming data request to determine the appropiate return responses.

For the **Endpoint URL**, you will need to enter your Kafka Client Amazon EC2 Instance PublicDNS. You can obtain your EC2 Instance Public DNS by navigating to the EC2 console. Here, select your client EC2 machine and look for Public IPv4 DNS and copy this. The endpoint URL should have the following format: 

`http://KafkaClientEC2InstancePublicDNS:8082/{proxy}`


<p align="center">
    <img src="images/APIs Integration.png" width="700" height="300"/>
</p>

> By creating a proxy resource with the **{proxy+}** parameter and the **ANY** method, you can provide your integration with access to all available resources.

## Installing Confluent package for REST proxy on EC2 client

To be able to consume data using MSK from the API we have just created, we will need to download some additional packages on a client EC2 machine, that will be used to communicate with the MSK cluster.

To install the REST proxy package run the following commands on your EC2 instance:

In [ ]:
sudo wget https://packages.confluent.io/archive/7.2/confluent-7.2.0.tar.gz
tar -xvzf confluent-7.2.0.tar.gz 

You should now be able to see a `confluent-7.2.0` directory on your EC2 instance. To configure the REST proxy to communicate with the desired MSK cluster, and to perform IAM authentication you first need to navigate to `confluent-7.2.0/etc/kafka-rest`. Inside here run the following command to modify the `kafka-rest.properties` file:

`nano kafka-rest.properties`

Firstly, you need to modify the `bootstrap.servers` and the `zookeeper.connect` variables in this file, with the corresponding **Boostrap server string** and **Plaintext Apache Zookeeper connection string** respectively (check-out the MSK Essential lesson to see how to obtain these). This will allow the REST proxy to connect to our MSK cluster.

Secondly, to surpass the IAM authentication of the MSK cluster, we will make use of the **IAM MSK authentication package** again (can be found here: https://github.com/aws/aws-msk-iam-auth). That means, you should add the following to your `kafka-rest.properties` file:

In [ ]:
# Identifies the SASL mechanism to use.
client.sasl.mechanism = AWS_MSK_IAM

# Binds SASL client implementation.
client.sasl.jaas.config = software.amazon.msk.auth.iam.IAMLoginModule required awsRoleArn="Your Access Role";

# Encapsulates constructing a SigV4 signature based on extracted credentials.
# The SASL client bound by "sasl.jaas.config" invokes this class.
client.sasl.client.callback.handler.class = software.amazon.msk.auth.iam.IAMClientCallbackHandler

Notice the difference from the Kafka `client.properties` file we set-up in the MSK Essentials lesson and this one. 
> To allow communication between the REST proxy and the cluster brokers, all configurations should be prefixed with client.

## Deploy API

Make note of the **Invoke URL** after deploying the API. Your external Kafka REST Proxy, which was exposed through API Gateway will look like: 

`https://YourAPIInvokeURL/test/topics/<AllYourTopics>` 

You will use this URL to send messages through API Gateway into your Kafka consumer.

## Starting the REST proxy

Before sending messages to the API, in order to make sure they are consumed in MSK, we need to start our REST proxy. To do this, first navigate to the `confluent-7.2.0/bin` folder, and then run the following command:

`./kafka-rest-start /home/ec2-user/confluent-7.2.0/etc/kafka-rest/kafka-rest.properties`

If everything went well, and your proxy is ready to received requests from the API, you should see a **INFO Server started, listening for requests...** in your EC2 console.

## API Repsonses in Python

Now we can use the Python requests library to test the API and obtain a response. Below, you have an example code structure for sending messages to an AWS API Gateway API using the **Invoke URL**.

In [ ]:
import requests
import json

example_df = {"index": 1, "name": "Maya", "age": 25, "role": "engineer"}

invoke_url = "https://YourAPIInvokeURL/YourDeploymentStage/topics/YourTopicName"
#To send JSON messages you need to follow this structure
payload = json.dumps({
    "records": [
        {
        #Data should be send as pairs of column_name:value, with different columns separated by commas       
        "value": {"index": example_df["index"], "name": example_df["name"], "age": example_df["age"], "role": example_df["role"]}
        }
    ]
})

headers = {'Content-Type': 'application/vnd.kafka.json.v2+json'}
response = requests.request("POST", invoke_url, headers=headers, data=payload)

To see whether the request was successfully processed we can `print(response.status_code)`, which should return a status 200, indicating success. Moreover, you should see these messages being consumed in our Kafka consumer.

## Conclusion
At this point, you should have a good understanding of:

* How to configure an API with Kafka REST Proxy Integration.
* How to send requests to a Kafka REST Proxy API using Python